# Data Profiling
## Team 8: Anthony, Sean, Gideon
## Team 9: Rohith, Sneha
## Team 10: Anmol, Nikita, Satya

### The first deliverable is a report of the 25 top selling products.
### The first column requires us to aggregate the 3 databases and then generate the top 25.

#### Database API taken from Anthony Ung's code for HW 4.
#### This API was modified so an instance may be created.

In [7]:
import sqlite3 as lite
from enum import Enum

'''
    This class provides one common point of interaction with my team's database.
    Everything that writes to the database uses this API.
'''
class db:

    class Options(Enum):
        RETURN_RESULTS = 1
        PRINT_RESULTS = 2
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, options=0):
        if (options.value & db.Options.RETURN_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            return results
            
        self.cur.execute(sql)

    def execute_sql_values(self, sql, values):
        self.cur.execute(sql, values)

    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

#### Connect to Team 9's Database and do a Demo Query

In [8]:
db_9 = db('../0_SD_Team_9/grocery_store.db')
db_9.connect()

In [10]:
sql = 'SELECT * FROM transactions LIMIT 10'

db_9.execute_sql(sql, options=db.Options.RETURN_RESULTS)

[('2024-01-01', 0, 42360001, 2.02, 1109, 92.5),
 ('2024-01-01', 0, 43403001, 3.2, 1151, 96),
 ('2024-01-01', 0, 44034001, 1.17, 629, 52.5),
 ('2024-01-01', 0, 43210001, 23.53, 527, 44),
 ('2024-01-01', 0, 43454001, 2.88, 74, 6.25),
 ('2024-01-01', 0, 42668001, 3.52, 74, 6.25),
 ('2024-01-01', 0, 43244001, 5.87, 74, 6.25),
 ('2024-01-01', 0, 42883001, 2.68, 74, 6.25),
 ('2024-01-01', 0, 43981001, 0.64, 74, 6.25),
 ('2024-01-01', 0, 43702001, 15.18, 74, 6.25)]

#### Connect to Team 8's Database and do a Demo Query

In [11]:
db_8 = db('../0_SD_Team_8/store_team_8.db')
db_8.connect()

In [12]:
sql = 'SELECT * FROM sales_transactions LIMIT 10'

db_8.execute_sql(sql, options=db.Options.RETURN_RESULTS)

[('2024-01-01', 1, 43365001, 1.07, 23, 2),
 ('2024-01-01', 1, 43018001, 13.19, 23, 2),
 ('2024-01-01', 1, 43083001, 2.68, 83, 7),
 ('2024-01-01', 1, 42779001, 13.19, 83, 7),
 ('2024-01-01', 1, 42238001, 1.2, 83, 7),
 ('2024-01-01', 1, 43104001, 1.99, 83, 7),
 ('2024-01-01', 1, 43482001, 4.79, 83, 7),
 ('2024-01-01', 1, 42303001, 1.43, 83, 7),
 ('2024-01-01', 1, 42864001, 3.0, 83, 7),
 ('2024-01-01', 1, 43761001, 1.76, 83, 7)]

#### Connect to Team 10's Database and do a Demo Query

In [13]:
db_10 = db('../0_SD_Team_10/grocery_team_10.db')
db_10.connect()

In [15]:
sql = 'SELECT * FROM sales_transactions LIMIT 10'

results = db_10.execute_sql(sql, options=db.Options.RETURN_RESULTS)
for row in results:
    print(row)

('20240101', 0, b'\x91\xd5\x88\x02\x00\x00\x00\x00', 4.59, 100000007, 8333324)
('20240101', 0, b'\xb9o\x8a\x02\x00\x00\x00\x00', 2.86, 100000007, 8333324)
('20240101', 0, b'q\x1c\x85\x02\x00\x00\x00\x00', 2.63, 100000007, 8333324)
('20240101', 1, b'\x11\x1a\x88\x02\x00\x00\x00\x00', 2.6, 100000007, 8333324)
('20240101', 1, b'\xe9\x84\x9a\x02\x00\x00\x00\x00', 1.52, 100000007, 8333324)
('20240101', 1, b'i\x94\x8d\x02\x00\x00\x00\x00', 1.75, 100000007, 8333324)
('20240101', 1, b'\xc1,\x8e\x02\x00\x00\x00\x00', 2.86, 100000007, 8333324)
('20240101', 2, b'A\xb2\x97\x02\x00\x00\x00\x00', 5.16, 100000007, 8333324)
('20240101', 2, b'\x99Q\x82\x02\x00\x00\x00\x00', 3.44, 100000007, 8333324)
('20240101', 2, b'QJ\x9a\x02\x00\x00\x00\x00', 0.66, 100000007, 8333324)
